In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as linalg
import torch
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
d=2        ## PC Num
n=5        ## sample Num

## 背景知识--MDS

假设 $m$ 个样本在原始空间的**距离矩阵** $D \in R^{m \times m}$，其 $i$ 行 $j$ 列的元素 $dist_{ij}$ 表示样本 $x_i$ 到 $x_j$ 的距离，多维缩放(Multiple Dimensional Scaling, MDS)的目标是获得样本在 $d'<m$ 维空间的表示，并且**任意样本在低维空间中的距离等于原始空间中的距离**（欧式距离/其它）



1. 内积矩阵$B$，其元素 $b_{ij} = -\frac{1}{2}(dist_{ij}^2-dist_{i-}^2-dist_{-j}^2+dist_{--}^2)$    （Gower’s centring）
    - $dist_{i-}^2=\frac{1}{m}\sum\limits_{j=1}^{m}dist_{ij}^2$
    - $dist_{-j}^2=\frac{1}{m}\sum\limits_{i=1}^{m}dist_{ij}^2$
    - $dist_{--}^2=\frac{1}{m^2}\sum\limits_{i=1}^{m}\sum\limits_{j=1}^{m}dist_{ij}^2$

2. 对内积矩阵$B$进行特征值分解，取 $\Lambda$ 为 $d'$ 个最大特征值构成的对角矩阵，$V$ 为相应的特征向量矩阵
3. 得到新矩阵 $\Lambda V^{1/2} \in R^{m \times d'}$，每行是一个样本的低维坐标 （Scaling 2）


## PCoA

(Numerical Ecology, ch9.3) 假设有一个Species丰度矩阵Y，每行代表一个样本，每列代表一个descriptor/feature

1. 计算距离矩阵D，本例中使用欧氏距离，事实上支持各种距离
2. Gower’s centring 得到 A
3. 对 A 分解特征值，得到的特征向量U进行Scaling 2后即Sample在PC空间的坐标

如果有负的特征值（可能由于欧氏距离不足以描述关系、或者由于处理缺失值的策略），需要修正D直到变成正数:（注意，修正后就不是原来的distance representation了）

(Method 1, p502) $D=\sqrt{D^2+2c_1}$；其中c1是原PCoA所得最负特征值的绝对值

(Method 2, p503) $D=D+c_2$；其中c2见当页描述


In [2]:
Y = np.matrix([[2,3,5,7,9],        ## n=5, feature=2   as in NumEco_PCA.ipynb
               [1,4,0,6,2]]).T


D = euclidean_distances(np.asarray(Y), np.asarray(Y)) 
D

array([[0.        , 3.16227766, 3.16227766, 7.07106781, 7.07106781],
       [3.16227766, 0.        , 4.47213595, 4.47213595, 6.32455532],
       [3.16227766, 4.47213595, 0.        , 6.32455532, 4.47213595],
       [7.07106781, 4.47213595, 6.32455532, 0.        , 4.47213595],
       [7.07106781, 6.32455532, 4.47213595, 4.47213595, 0.        ]])

In [3]:
A = - D * D / 2 
A = A - np.array([A.mean(0)] *n).T - np.array([A.mean(1)] *n) + A.mean()         ## Gower’s centring
# A = np.matrix(A)
A

array([[ 12.8,   4.8,   4.8, -11.2, -11.2],
       [  4.8,   6.8,  -3.2,   0.8,  -9.2],
       [  4.8,  -3.2,   6.8,  -9.2,   0.8],
       [-11.2,   0.8,  -9.2,  14.8,   4.8],
       [-11.2,  -9.2,   0.8,   4.8,  14.8]])

In [4]:
L, U = linalg.eigh(A)
L = L[::-1]                  ## np.argsort(L)[::-1]  to descending order
U = U[:,::-1]               

L = L[:d]
U = U[:,:d]

L,U       

(array([36., 20.]),
 array([[-5.96284794e-01, -4.36645107e-16],
        [-2.23606798e-01, -5.00000000e-01],
        [-2.23606798e-01,  5.00000000e-01],
        [ 5.21749195e-01, -5.00000000e-01],
        [ 5.21749195e-01,  5.00000000e-01]]))

In [5]:
L/(n-1)                  ## eigvalues of corresponding PCA

array([9., 5.])

### Scaling 2

In [6]:
U_ = U @ np.diag(np.sqrt(L))      ## samples 在 PC轴中的坐标
U_

array([[-3.57770876e+00, -1.95273628e-15],
       [-1.34164079e+00, -2.23606798e+00],
       [-1.34164079e+00,  2.23606798e+00],
       [ 3.13049517e+00, -2.23606798e+00],
       [ 3.13049517e+00,  2.23606798e+00]])

## 关于Vegan

```R
library(vegan)
Y = t(matrix( c(2,3,5,7,9,1,4,0,6,2), ncol = 5,byrow = TRUE))
D <- dist(Y)
res <- cmdscale(D,k=2,eig = TRUE)
```

结果：
```R
> res$points
          [,1]          [,2]
[1,] -3.577709  1.569229e-15
[2,] -1.341641  2.236068e+00
[3,] -1.341641 -2.236068e+00
[4,]  3.130495  2.236068e+00
[5,]  3.130495 -2.236068e+00
> res$eig
[1]  3.600000e+01  2.000000e+01  6.445720e-15 -3.552714e-15 -9.394806e-15
```

